# 层和块
回顾一下多层感知机

In [1]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)

tensor([[ 0.1473,  0.1754, -0.2293, -0.2209,  0.1558,  0.0225, -0.0283, -0.2109,
         -0.1728,  0.1778],
        [ 0.0129,  0.1065, -0.0837, -0.3443,  0.0648, -0.0671, -0.1298, -0.1763,
          0.0315,  0.0939]], grad_fn=<AddmmBackward0>)

# 自定义块

In [5]:
class MLP(nn.Module):
    # 用模型参数声明层。这里，我们声明两个全连接的层
    def __init__(self):
        # 调用MLP的父类Module的构造函数来执行必要的初始化。
        # 这样，在类实例化时也可以指定其他函数参数，例如模型参数params（稍后将介绍）
        super().__init__()
        self.hidden = nn.Linear(20, 256)  # 隐藏层
        self.out = nn.Linear(256, 10)  # 输出层

    # 定义模型的前向传播，即如何根据输入X返回所需的模型输出
    def forward(self, X):
        # 注意，这里我们使用ReLU的函数版本，其在nn.functional模块中定义。
        return self.out(F.relu(self.hidden(X)))

# 实例化多层感知机的层，然后在每次调用前向传播函数时调用这些层
注意，除非我们实现一个新的运算符， 否则我们不必担心反向传播函数或参数初始化， 系统将自动生成这些。

In [6]:
net = MLP()
net(X)

tensor([[ 0.0306, -0.1433, -0.0807,  0.0844, -0.1217, -0.0908, -0.1556,  0.1653,
         -0.0759,  0.2057],
        [ 0.0296, -0.0698,  0.0272,  0.2041, -0.1326, -0.1981, -0.1730,  0.1910,
         -0.1617,  0.2164]], grad_fn=<AddmmBackward0>)

# 顺序块

In [9]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            # 这里，module是Module子类的一个实例。我们把它保存在'Module'类的成员
            # 变量_modules中。_module的类型是OrderedDict
            self._modules[str(idx)] = module

    def forward(self, X):
        # OrderedDict保证了按照成员添加的顺序遍历它们
        for block in self._modules.values():
            X = block(X)
        return X

net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[-0.1613, -0.0508, -0.0959,  0.0478, -0.0502,  0.0851, -0.0704,  0.1960,
         -0.1144,  0.1668],
        [ 0.0215, -0.0571, -0.0407,  0.1380, -0.1630, -0.0048, -0.0315,  0.0553,
         -0.0903, -0.0047]], grad_fn=<AddmmBackward0>)

# 在前向传播函数中执行代码

In [11]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # 不计算梯度的随机权重参数。因此其在训练期间保持不变
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        # 使用创建的常量参数以及relu和mm函数
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        # 复用全连接层。这相当于两个全连接层共享参数
        X = self.linear(X)
        # 控制流
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

net = FixedHiddenMLP()
net(X)

tensor(0.0406, grad_fn=<SumBackward0>)

#混合搭配各种组合块的方法

In [12]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(0.1491, grad_fn=<SumBackward0>)